In [1]:
from pyspark.ml import PipelineModel
from pyspark.sql.functions import col, lit, to_date
from pyspark.sql.types import IntegerType

model_path = "gs://model_stored_for_pred_forecast/daily_prediction/"
model = PipelineModel.load(model_path)

In [2]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

In [3]:
# Get November data (for moving avg calc)
query_nov = """
            CREATE OR REPLACE TABLE `lively-encoder-448916-d5.nyc_subway.dec_first_half` AS
            SELECT
            transit_date,
            transit_mode_index,
            station_complex_index,
            borough_index,
            payment_method_index,
            ridership AS actual_ridership,
            ridership AS prediction,
            day_of_week,
            day_of_week_sin,
            day_of_week_cos,
            week_of_month,
            week_of_month_sin,
            week_of_month_cos,
            ridership_lag_1,
            ridership_lag_2,
            ridership_lag_3,
            ridership_lag_4,
            ridership_lag_5,
            ridership_lag_6,
            ridership_lag_7,
            ridership_lag_8,
            ridership_lag_9,
            ridership_lag_10,
            ridership_lag_11,
            ridership_lag_12,
            ridership_lag_13,
            ridership_lag_14,
            ridership_lag_15,
            ridership_lag_16,
            ridership_lag_17,
            ridership_lag_18,
            ridership_lag_19,
            ridership_lag_20,
            ridership_lag_21,
            ridership_lag_22,
            ridership_lag_23,
            ridership_lag_24,
            ridership_lag_25,
            ridership_lag_26,
            ridership_lag_27,
            ridership_lag_28,
            ridership_lag_29,
            ridership_lag_30,
            ridership_7d_mv,
            day_of_week_7d_mv,
            week_of_month_7d_mv,
            day_of_week_sin_7d_mv,
            day_of_week_cos_7d_mv,
            week_of_month_sin_7d_mv,
            week_of_month_cos_7d_mv,
            ridership_30d_mv,
            day_of_week_30d_mv,
            week_of_month_30d_mv,
            day_of_week_sin_30d_mv,
            day_of_week_cos_30d_mv,
            week_of_month_sin_30d_mv,
            week_of_month_cos_30d_mv
            FROM `lively-encoder-448916-d5.nyc_subway.date_model`
            WHERE transit_date >= '2024-12-01' AND transit_date <= '2024-12-15';
            """

In [4]:
# Create December data (to extract actualridership values)
query_dec = """
            CREATE OR REPLACE TABLE `lively-encoder-448916-d5.nyc_subway.dec_second_half` AS
            SELECT
            transit_date,
            transit_mode_index,
            station_complex_index,
            borough_index,
            payment_method_index,
            ridership AS actual_ridership,
            day_of_week,
            day_of_week_sin,
            day_of_week_cos,
            week_of_month,
            week_of_month_sin,
            week_of_month_cos,
            ridership_lag_1,
            ridership_lag_2,
            ridership_lag_3,
            ridership_lag_4,
            ridership_lag_5,
            ridership_lag_6,
            ridership_lag_7,
            ridership_lag_8,
            ridership_lag_9,
            ridership_lag_10,
            ridership_lag_11,
            ridership_lag_12,
            ridership_lag_13,
            ridership_lag_14,
            ridership_lag_15,
            ridership_lag_16,
            ridership_lag_17,
            ridership_lag_18,
            ridership_lag_19,
            ridership_lag_20,
            ridership_lag_21,
            ridership_lag_22,
            ridership_lag_23,
            ridership_lag_24,
            ridership_lag_25,
            ridership_lag_26,
            ridership_lag_27,
            ridership_lag_28,
            ridership_lag_29,
            ridership_lag_30,
            ridership_7d_mv,
            day_of_week_7d_mv,
            week_of_month_7d_mv,
            day_of_week_sin_7d_mv,
            day_of_week_cos_7d_mv,
            week_of_month_sin_7d_mv,
            week_of_month_cos_7d_mv,
            ridership_30d_mv,
            day_of_week_30d_mv,
            week_of_month_30d_mv,
            day_of_week_sin_30d_mv,
            day_of_week_cos_30d_mv,
            week_of_month_sin_30d_mv,
            week_of_month_cos_30d_mv
            FROM `lively-encoder-448916-d5.nyc_subway.date_model`
            WHERE transit_date >= '2024-12-16' AND transit_date <= '2024-12-31';
            """

In [5]:
# Calculate lags
query_lags = """
                CREATE OR REPLACE TABLE `lively-encoder-448916-d5.nyc_subway.dec_lags`
                AS
                   SELECT
                       (SELECT MAX(transit_date)+1 FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`) AS transit_date,
                       transit_mode_index, station_complex_index, borough_index, payment_method_index,
                       prediction as ridership_lag_1,
                       ridership_lag_1 as ridership_lag_2,
                       ridership_lag_2 as ridership_lag_3,
                       ridership_lag_3 as ridership_lag_4,
                       ridership_lag_4 as ridership_lag_5,
                       ridership_lag_5 as ridership_lag_6,
                       ridership_lag_6 as ridership_lag_7,
                       ridership_lag_7 as ridership_lag_8,
                       ridership_lag_8 as ridership_lag_9,
                       ridership_lag_9 as ridership_lag_10,
                       ridership_lag_10 as ridership_lag_11,
                       ridership_lag_11 as ridership_lag_12,
                       ridership_lag_12 as ridership_lag_13,
                       ridership_lag_13 as ridership_lag_14,
                       ridership_lag_14 as ridership_lag_15,
                       ridership_lag_15 as ridership_lag_16,
                       ridership_lag_16 as ridership_lag_17,
                       ridership_lag_17 as ridership_lag_18,
                       ridership_lag_18 as ridership_lag_19,
                       ridership_lag_19 as ridership_lag_20,
                       ridership_lag_20 as ridership_lag_21,
                       ridership_lag_21 as ridership_lag_22,
                       ridership_lag_22 as ridership_lag_23,
                       ridership_lag_23 as ridership_lag_24,
                       ridership_lag_24 as ridership_lag_25,
                       ridership_lag_25 as ridership_lag_26,
                       ridership_lag_26 as ridership_lag_27,
                       ridership_lag_27 as ridership_lag_28,
                       ridership_lag_28 as ridership_lag_29,
                       ridership_lag_29 as ridership_lag_30
                   FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`
                   WHERE transit_date = (SELECT MAX(transit_date) FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`)
                   LIMIT 5000
                """

In [6]:
# Calcuate moving average
query_ma = """
            CREATE OR REPLACE TABLE `lively-encoder-448916-d5.nyc_subway.dec_ma`
               AS (
               WITH ridership_moving_avg AS (
                   SELECT
                       transit_date,
                       transit_mode_index,
                       station_complex_index,
                       borough_index,
                       payment_method_index,
                       -- 7-day moving average
                       AVG(prediction) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 6 PRECEDING AND 0 PRECEDING
                       ) AS ridership_7d_mv,
                       AVG(day_of_week) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 6 PRECEDING AND 0 PRECEDING
                       ) AS day_of_week_7d_mv,
                       AVG(week_of_month) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 6 PRECEDING AND 0 PRECEDING
                       ) AS week_of_month_7d_mv,
                       AVG(day_of_week_sin) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 6 PRECEDING AND 0 PRECEDING
                       ) AS day_of_week_sin_7d_mv,
                       AVG(day_of_week_cos) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 6 PRECEDING AND 0 PRECEDING
                       ) AS day_of_week_cos_7d_mv,
                       AVG(week_of_month_sin) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 6 PRECEDING AND 0 PRECEDING
                       ) AS week_of_month_sin_7d_mv,
                       AVG(week_of_month_cos) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 6 PRECEDING AND 0 PRECEDING
                       ) AS week_of_month_cos_7d_mv,
                       -- 30-day moving average
                       AVG(prediction) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 29 PRECEDING AND 0 PRECEDING
                       ) AS ridership_30d_mv,
                       AVG(day_of_week) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 29 PRECEDING AND 0 PRECEDING
                       ) AS day_of_week_30d_mv,
                       AVG(week_of_month) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 29 PRECEDING AND 0 PRECEDING
                       ) AS week_of_month_30d_mv,
                       AVG(day_of_week_sin) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 29 PRECEDING AND 0 PRECEDING
                       ) AS day_of_week_sin_30d_mv,
                       AVG(day_of_week_cos) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 29 PRECEDING AND 0 PRECEDING
                       ) AS day_of_week_cos_30d_mv,
                       AVG(week_of_month_sin) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 29 PRECEDING AND 0 PRECEDING
                       ) AS week_of_month_sin_30d_mv,
                       AVG(week_of_month_cos) OVER (
                           PARTITION BY station_complex_index, transit_mode_index, borough_index, payment_method_index
                           ORDER BY transit_date
                           ROWS BETWEEN 29 PRECEDING AND 0 PRECEDING
                       ) AS week_of_month_cos_30d_mv
                   FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`
                   WHERE transit_date BETWEEN
                       (SELECT MAX(transit_date) - 29 FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`)
                       AND
                       (SELECT MAX(transit_date) FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`)
               )
               SELECT
                   transit_date + 1 AS next_transit_date,
                   *
               FROM
                   ridership_moving_avg
               WHERE transit_date = (SELECT MAX(transit_date) FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`)
               ORDER BY transit_date DESC
               LIMIT 5000
               );
            """

In [7]:
# Combine lags and ma tables to create input df for model
query_input = """
                CREATE OR REPLACE TABLE `lively-encoder-448916-d5.nyc_subway.dec_input`
                   AS (
                   SELECT
                       l.transit_date,
                       l.transit_mode_index,
                       l.station_complex_index,
                       l.borough_index,
                       l.payment_method_index,
                       EXTRACT(DAYOFWEEK FROM l.transit_date) AS day_of_week,
                       SIN(2 * ACOS(-1) * EXTRACT(DAYOFWEEK FROM l.transit_date) / 7) AS day_of_week_sin,
                       COS(2 * ACOS(-1) * EXTRACT(DAYOFWEEK FROM l.transit_date) / 7) AS day_of_week_cos,
                       CEIL(EXTRACT(DAY FROM l.transit_date) / 7) AS week_of_month,
                       SIN(2 * ACOS(-1) * CEIL(EXTRACT(DAY FROM l.transit_date) / 7) / 5) AS week_of_month_sin,
                       COS(2 * ACOS(-1) * CEIL(EXTRACT(DAY FROM l.transit_date) / 7) / 5) AS week_of_month_cos,
                       l.ridership_lag_1, l.ridership_lag_2, l.ridership_lag_3, l.ridership_lag_4, l.ridership_lag_5,
                       l.ridership_lag_6, l.ridership_lag_7, l.ridership_lag_8, l.ridership_lag_9, l.ridership_lag_10,
                       l.ridership_lag_11, l.ridership_lag_12, l.ridership_lag_13, l.ridership_lag_14, l.ridership_lag_15,
                       l.ridership_lag_16, l.ridership_lag_17, l.ridership_lag_18, l.ridership_lag_19, l.ridership_lag_20,
                       l.ridership_lag_21, l.ridership_lag_22, l.ridership_lag_23, l.ridership_lag_24, l.ridership_lag_25,
                       l.ridership_lag_26, l.ridership_lag_27, l.ridership_lag_28, l.ridership_lag_29, l.ridership_lag_30,
                       mv.ridership_7d_mv,
                       mv.day_of_week_7d_mv, mv.week_of_month_7d_mv,
                       mv.day_of_week_sin_7d_mv, mv.day_of_week_cos_7d_mv,
                       mv.week_of_month_sin_7d_mv, mv.week_of_month_cos_7d_mv,
                       mv.ridership_30d_mv,
                       mv.day_of_week_30d_mv, mv.week_of_month_30d_mv,
                       mv.day_of_week_sin_30d_mv, mv.day_of_week_cos_30d_mv,
                       mv.week_of_month_sin_30d_mv, mv.week_of_month_cos_30d_mv
                   FROM `lively-encoder-448916-d5.nyc_subway.dec_lags` l
                   INNER JOIN `lively-encoder-448916-d5.nyc_subway.dec_ma` mv
                   ON l.transit_mode_index = mv.transit_mode_index
                   AND l.station_complex_index = mv.station_complex_index
                   AND l.borough_index = mv.borough_index
                   AND l.payment_method_index = mv.payment_method_index
                   );
                """

In [8]:
query_buffer = """
                CREATE OR REPLACE TABLE `lively-encoder-448916-d5.nyc_subway.dec_buffer` AS
                SELECT
                  -- Columns from dec_input
                  dfi.transit_date,
                  dfi.transit_mode_index,
                  dfi.station_complex_index,
                  dfi.borough_index,
                  dfi.payment_method_index,
                  -- Column from dec_second_half
                  dd.actual_ridership,
                  -- Column from dec_output
                  dfo.prediction,
                  -- Other columns from dec_input
                  dfi.day_of_week,
                  dfi.day_of_week_sin,
                  dfi.day_of_week_cos,
                  dfi.week_of_month,
                  dfi.week_of_month_sin,
                  dfi.week_of_month_cos,
                  dfi.ridership_lag_1,
                  dfi.ridership_lag_2,
                  dfi.ridership_lag_3,
                  dfi.ridership_lag_4,
                  dfi.ridership_lag_5,
                  dfi.ridership_lag_6,
                  dfi.ridership_lag_7,
                  dfi.ridership_lag_8,
                  dfi.ridership_lag_9,
                  dfi.ridership_lag_10,
                  dfi.ridership_lag_11,
                  dfi.ridership_lag_12,
                  dfi.ridership_lag_13,
                  dfi.ridership_lag_14,
                  dfi.ridership_lag_15,
                  dfi.ridership_lag_16,
                  dfi.ridership_lag_17,
                  dfi.ridership_lag_18,
                  dfi.ridership_lag_19,
                  dfi.ridership_lag_20,
                  dfi.ridership_lag_21,
                  dfi.ridership_lag_22,
                  dfi.ridership_lag_23,
                  dfi.ridership_lag_24,
                  dfi.ridership_lag_25,
                  dfi.ridership_lag_26,
                  dfi.ridership_lag_27,
                  dfi.ridership_lag_28,
                  dfi.ridership_lag_29,
                  dfi.ridership_lag_30,
                  dfi.ridership_7d_mv,
                  dfi.day_of_week_7d_mv,
                  dfi.week_of_month_7d_mv,
                  dfi.day_of_week_sin_7d_mv,
                  dfi.day_of_week_cos_7d_mv,
                  dfi.week_of_month_sin_7d_mv,
                  dfi.week_of_month_cos_7d_mv,
                  dfi.ridership_30d_mv,
                  dfi.day_of_week_30d_mv,
                  dfi.week_of_month_30d_mv,
                  dfi.day_of_week_sin_30d_mv,
                  dfi.day_of_week_cos_30d_mv,
                  dfi.week_of_month_sin_30d_mv,
                  dfi.week_of_month_cos_30d_mv

                FROM `lively-encoder-448916-d5.nyc_subway.dec_input` AS dfi
                INNER JOIN `lively-encoder-448916-d5.nyc_subway.dec_output` AS dfo
                  ON dfi.transit_date = dfo.transit_date
                  AND dfi.transit_mode_index = dfo.transit_mode_index
                  AND dfi.station_complex_index = dfo.station_complex_index
                  AND dfi.borough_index = dfo.borough_index
                  AND dfi.payment_method_index = dfo.payment_method_index

                INNER JOIN `lively-encoder-448916-d5.nyc_subway.dec_second_half` AS dd
                  ON dfi.transit_date = dd.transit_date
                  AND dfi.transit_mode_index = dd.transit_mode_index
                  AND dfi.station_complex_index = dd.station_complex_index
                  AND dfi.borough_index = dd.borough_index
                  AND dfi.payment_method_index = dd.payment_method_index;
                """

In [9]:
query_union = """
                CREATE OR REPLACE TABLE `lively-encoder-448916-d5.nyc_subway.dec_first_half` AS
                SELECT * FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`
                UNION ALL
                SELECT * FROM `lively-encoder-448916-d5.nyc_subway.dec_buffer`;
                """

In [10]:
# Create November and December tables
client.query(query_nov).result()
client.query(query_dec).result()

In [14]:
import time

for i in range(1, 10):
    
    # Add lags, mv, combine for input
    client.query(query_lags).result()
    client.query(query_ma).result()
    client.query(query_input).result()
    time.sleep(4)
    
    # Read input data from BigQuery table into a Spark DataFrame
    input_daily_df = spark.read \
        .format("bigquery") \
        .option("table", "lively-encoder-448916-d5.nyc_subway.dec_input") \
        .option("parentProject", "lively-encoder-448916-d5") \
        .load()

    # Extract the first value of the `transit_date` column
    df_transit_date = input_daily_df.select("transit_date").first()["transit_date"]
    print(df_transit_date)

    # Drop the `transit_date` column from the DataFrame before inputting to the model
    input_daily_df = input_daily_df.drop("transit_date")

    # Get predictions
    output_predictions = model.transform(input_daily_df)
    output_predictions = output_predictions.withColumn("transit_date", to_date(lit(df_transit_date)))
    output_predictions = output_predictions.withColumn("prediction", col("prediction").cast(IntegerType()))
    output_predictions = output_predictions.select('transit_date', 'transit_mode_index', 'station_complex_index', 'borough_index', 'payment_method_index', 'prediction')

    # Write predictions
    output_predictions.write.format("bigquery") \
        .option("table", "lively-encoder-448916-d5.nyc_subway.dec_output") \
        .option("temporaryGcsBucket", "temp_dec_forecast_bucket") \
        .mode("overwrite") \
        .save()
    
    time.sleep(4)
    # Update recent_dates_df
    client.query(query_buffer).result()
    client.query(query_union).result()
    
    time.sleep(1)
    
    

2024-12-23


2024-12-24


2024-12-25


2024-12-26


2024-12-27


2024-12-28


2024-12-29


2024-12-30


2024-12-31


In [15]:
query_dec_avp = """
                CREATE OR REPLACE TABLE `lively-encoder-448916-d5.nyc_subway.dec_avp` AS
                SELECT
                    transit_date, transit_mode_index,
                    station_complex_index, borough_index,
                    payment_method_index,
                    actual_ridership, prediction
                FROM `lively-encoder-448916-d5.nyc_subway.dec_first_half`
                WHERE transit_date > '2024-11-15'
                ORDER BY transit_date DESC;
                """
client.query(query_dec_avp).result()

In [16]:
dec_avp = spark.read \
                    .format("bigquery") \
                    .option("table", "lively-encoder-448916-d5.nyc_subway.dec_avp") \
                    .option("parentProject", "lively-encoder-448916-d5") \
                    .load()

In [17]:
from pyspark.sql.functions import col, lit, create_map, round, to_date

# Map encoded categorical columns to their original values
payment_method_mapping = {1: 'metrocard', 2: 'omny'}
transit_mode_mapping = {1: 'staten_island_railway', 2: 'subway', 3: 'tram'}
borough_mapping = {1: 'Bronx', 2: 'Brooklyn', 3: 'Manhattan', 4: 'Queens', 5: 'Staten Island'}

station_df = spark.read.csv("gs://bucket_jars/station_complex_csv/station_complex.csv", header=True)
station_df = station_df.withColumn("station_complex_index", col("station_complex_index").cast("int"))
station_df = station_df.select("station_complex_index", "station_complex")
# station_df.printSchema()

station_map = station_df.rdd.collectAsMap()
# print(station_mapping)
station_mapping = [
    item for pair in station_map.items() for item in (lit(pair[0]), lit(pair[1]))
]

# Create mapping expressions
payment_method_expr = create_map([lit(k) for pair in payment_method_mapping.items() for k in pair])
transit_mode_expr = create_map([lit(k) for pair in transit_mode_mapping.items() for k in pair])
borough_expr = create_map([lit(k) for pair in borough_mapping.items() for k in pair])
station_complex_expr = create_map(*station_mapping)

# Apply the mappings
dec_avp_mapped = dec_avp.withColumn(
                        "payment_method", 
                        payment_method_expr[col("payment_method_index").cast("int")]
                    ).withColumn(
                        "transit_mode", 
                        transit_mode_expr[col("transit_mode_index").cast("int")]
                    ).withColumn(
                        "borough", 
                        borough_expr[col("borough_index").cast("int")]
                    ).withColumn(
                        "station_complex", 
                        station_complex_expr[col("station_complex_index").cast("int")]
                    )

In [ ]:
# dec_avp_mapped.show(5)

In [18]:
dec_avp_mapped = dec_avp_mapped.select('transit_date', 'transit_mode', 'station_complex', 'borough', 'payment_method', 'actual_ridership', 'prediction')
dec_avp_mapped.show(3)

+------------+------------+--------------------+---------+--------------+----------------+----------+
|transit_date|transit_mode|     station_complex|  borough|payment_method|actual_ridership|prediction|
+------------+------------+--------------------+---------+--------------+----------------+----------+
|  2024-12-31|      subway|        Astor Pl (6)|Manhattan|     metrocard|            2411|      3300|
|  2024-12-31|      subway|         28 St (R,W)|Manhattan|     metrocard|            2113|      2842|
|  2024-12-31|      subway|90 St-Elmhurst Av...|   Queens|     metrocard|            5386|      6833|
+------------+------------+--------------------+---------+--------------+----------------+----------+
only showing top 3 rows



In [19]:
# Write forecast
dec_avp_mapped.write.format("bigquery") \
    .option("table", "lively-encoder-448916-d5.nyc_subway.dec_avp") \
    .option("temporaryGcsBucket", "temp_dec_forecast_bucket") \
    .mode("overwrite") \
    .save()